In [12]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input 
import numpy as np
import os
from tqdm import tqdm
import pickle
from sklearn.neighbors import NearestNeighbors
import cv2


# Creating Model

In [2]:
model= ResNet50(weights='imagenet',include_top=False,input_shape=(224,224,3))
model.trainable=False
model=tf.keras.Sequential([
model,
GlobalMaxPooling2D()
])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_max_pooling2d (Globa  (None, 2048)             0         
 lMaxPooling2D)                                                  
                                                                 
Total params: 23,587,712
Trainable params: 0
Non-trainable params: 23,587,712
_________________________________________________________________


# Extracting features from data

In [3]:
def extract_features(img_path,model):
    img=image.load_img(img_path,target_size=(224,224))
    img_array=image.img_to_array(img)
    expanded_img_array=np.expand_dims(img_array,axis=0)
    preprocessed_img=preprocess_input(expanded_img_array)
    result=model.predict(preprocessed_img).flatten()
    norm_result=result/np.linalg.norm(result)
    return norm_result


In [4]:
filenames1=[]
for file in os.listdir('D:\Programs\Python\ML_notebooks\Fashion_recommendation\images'):
    filenames1.append(os.path.join('D:\Programs\Python\ML_notebooks\Fashion_recommendation\images',file))

In [6]:
# feature_list=[]
# for file in tqdm(filenames):
#     feature_list.append(extract_features(file,model))

# pickle.dump(feature_list,open('feature_list_embeddings.pkl','wb'))
# pickle.dump(filenames,open('filenames.pkl','wb'))

In [5]:
feature_list1=np.array(pickle.load(open('D:\Programs\Python\ML_notebooks\Fashion_recommendation/feature_list_embeddings.pkl','rb')))
print((feature_list1).shape)

(36000, 2048)


In [6]:
filenames=pickle.load(open('D:\Programs\Python\ML_notebooks\Fashion_recommendation/filenames.pkl','rb'))


# Testing

In [7]:
img=image.load_img('C:/Users\91901\Downloads\My_pic.jpeg',target_size=(224,224))
img_array1=image.img_to_array(img)
expanded_img_array1=np.expand_dims(img_array1,axis=0)
preprocessed_img1=preprocess_input(expanded_img_array1)
result1=model.predict(preprocessed_img1).flatten()
norm_result1=result1/np.linalg.norm(result1)

1/1 [==============================] - 1s 1s/step


In [8]:
neighbours=NearestNeighbors(n_neighbors=5,algorithm='brute',metric='euclidean')
neighbours.fit(feature_list1)
distance, indices=neighbours.kneighbors([norm_result1])
print(indices)

[[11234 34286 25421  2712 25292]]


In [11]:
# # Load the pkl file
# with open ("D:\Programs\Python\ML_notebooks\Fashion_recommendation/filenames.pkl", "rb") as f:
#     filenames = pickle.load (f)

# # Define the new path
# new_path = "D:\Programs\Python\ML_notebooks\Fashion_recommendation\images"

# # Loop over the files and change their paths
# for file in tqdm(filenames):
#     # Get the current path
#     old_path = '/kaggle/input/fashion-product-images-dataset/fashion-dataset/images/' + file
#     # Get the file name from the path
#     file_name = os.path.basename(old_path)
#     # Create a new list of modified paths using list comprehension
#     new_filenames = [os.path.join(new_path, os.path.basename(file)) for file in filenames]

# # Save the modified pkl file
# with open ("filenames.pkl", "wb") as f:
#     pickle.dump (new_filenames, f) 


In [11]:
for file in indices[0]:
    tmp_img=cv2.imread(filenames[file])
    cv2.imshow('output',cv2.resize(tmp_img,(512,512)))
    cv2.waitKey(0)